In [ ]:
from taucmdr.analysis.analyses.profile_barplot import ProfileBarPlotVisualizer
from taucmdr.model.trial import Trial
from taucmdr.cf.storage.levels import PROJECT_STORAGE
from taucmdr.data.tauprofile import TauProfile

def trials_to_patch_lists(trials, metric):
    import operator
    from collections import defaultdict
    import pandas as pd
    import numpy as np
    from taucmdr.gui.color import ColorMapping
    from bokeh.models import ColumnDataSource
    sum_times = defaultdict(lambda: pd.DataFrame([], columns=[metric], dtype=np.float64))
    timer_names = set()
    # We need to get:
    #     - Names for all of the timers in all of the trials (as we need a line for each timer)
    #     - The total times for each trial across all processors
    for trial in trials:
        trial_num = trial['number']
        trial_data = trial.get_data()
        indices = TauProfile.indices(trial_data)
        num_processors = len(indices)
        trial_total_time = 0
        trial_times = pd.DataFrame([], columns=[metric], dtype=np.float64)
        for n, c, t in indices:
            df = trial_data[n][c][t].interval_data()[[metric]].loc[trial_num, n, c, t]
            trial_times = trial_times.add(df, fill_value=0.0)
        for name, value in trial_times.itertuples():
            timer_names.add(name)
        # If there are multiple trials with the same num_processors, we aggregate them
        sum_times[num_processors] = sum_times[num_processors].add(trial_times, fill_value=0.0)
    # Convert the times to percentages
    for num_processors in sum_times.keys():
        sum_times[num_processors] = (sum_times[num_processors]/(sum_times[num_processors].sum()))*100.0
    # Now we need to convert the percentages to 'patches' for Bokeh to render.
    # We need, for each timer, a list of x- and y-coordinates for the shape to be drawn.
    # From x = 1 to num_processors, we trace out the line with y, then drop down to the 
    # previous line and draw it from x = num_processors to 1.
    mapping = ColorMapping()
    raw_xs = []
    raw_ys = []
    colors = []
    names = []
    sorted_sum_times = sorted(sum_times.items())
    for timer_name in timer_names:
        timer_xs = []
        timer_ys = []
        for num_processors, df in sorted_sum_times:
            timer_xs.append(num_processors)
            timer_ys.append(float(df.loc[timer_name]))
        raw_xs.append(timer_xs)
        raw_ys.append(timer_ys)
        colors.append(mapping[timer_name])
        names.append(timer_name)
    # Postprocess the y values to stack the lines
    ys = []
    last = [0.0] * len(sorted_sum_times)
    for row in raw_ys:
        processed_row = map(operator.add, row, last)
        return_row = last[::-1]
        last = processed_row
        ys.append(processed_row + return_row)
    # Postprocess the x values for the return
    xs = []
    for row in raw_xs:
        xs.append(row + row[::-1])
    # Package everything into a data source
    data_source = ColumnDataSource(dict(x=xs, y=ys, color=colors, name=names))
    return data_source

        
        
        
def show_runtime_breakdown(trials, metric):
    from taucmdr.gui.interaction import InteractivePlotHandler
    from taucmdr import logger
    from bokeh.plotting import figure
    from bokeh.models.glyphs import HBar
    from bokeh.io import output_notebook
    from bokeh.models import LabelSet

    logger.set_log_level('WARN')
    output_notebook(hide_banner=True)

    def build_runtime_breakdown(trials, metric):
        title = trials[0].populate('experiment')['name']
        patch_lists = trials_to_patch_lists(trials, metric)
        fig = figure(plot_width=80, plot_height=40, output_backend="webgl", toolbar_location="right", title=title)
        fig.xaxis.axis_label = 'Number of Processors'
        fig.patches("x", "y", fill_color="color", line_color="color", alpha=0.9, source=patch_lists)
        return fig

    breakdown = build_runtime_breakdown(trials, metric)
    plot = InteractivePlotHandler(breakdown, tooltips=[("Timer", "@name")])
    plot.show()

show_runtime_breakdown([Trial.controller(PROJECT_STORAGE).search_hash("0a55be02ebaaa2ffcba5e80ad90662b4059883d5")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("91e392b88730cbc539555ef59f48d9f2d146c0e5")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("af0282a2f0603f9b96d106ca4ddda40082e6efa2")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("883c29fa41523f024578897f26aa98d9ccf93872")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("828374bf2d1549887554578766717531c5b9f091")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("061425ff2fca940c8918ebd325d95e522b659181")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("7284c749969c08de05d795b3ba334c2ff5075bb9")[0],
                        Trial.controller(PROJECT_STORAGE).search_hash("8629cf5864445aece7f16f53f7456dc5fcbf9f3d")[0]], "Exclusive")